# Courier

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [ ]:
courier_dimension = pd.read_sql_table("clientes_mensajeroaquitoy", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)

In [4]:
courier_dimension.head(1)

,id,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id,token_Firebase,url_foto
0,1,2,True,NaT,NaT,NaN,310-300000,13.0,None,http:


In [ ]:
city.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [ ]:
courier_dimension = courier_dimension.merge(
		city, left_on="ciudad_operacion_id", right_on="ciudad_id", how="left"
)
courier_dimension.head(1)

,id,user_id,activo,fecha_entrada,fecha_salida,salario,telefono,ciudad_operacion_id,token_Firebase,url_foto,ciudad_id,nombre,departamento_id
0,1,2,True,NaT,NaT,NaN,310-300000,13.0,None,http:,13.0,ACOPI YUMBO,1.0


In [ ]:
courier_dimension.rename(
	columns={
		"nombre": "courier_city"
	}, inplace=True
)

In [8]:
courier_dimension = courier_dimension[["courier_city"]]

In [9]:
courier_dimension.reset_index(inplace=True)
courier_dimension.rename(columns={ "index": "courier_id" }, inplace=True)
courier_dimension.set_index("courier_id", inplace=True)

In [10]:
courier_dimension.head(1)

,courier_city
courier_id,
0,ACOPI YUMBO


## Load

In [11]:
courier_dimension.to_sql(
	"COURIER_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

50